In [ ]:
# ENV (NLP)

# 0. Read Source Data & Preprocessing

In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('./data/3415_All_STORIES_Updated_ver02.xlsx', index_col = None, header = 0)

In [3]:
# Function to extract story content after the names

def extract_story(text):
    # Step 1: Split the text by line breaks
    lines = text.strip().split('\n')

    # Step 2: Detect the start of the story (the first longer segment, which should be a complete sentence)
    story_started = False
    story_lines = []
    
    for line in lines:
        # If the line contains more than a few words (let's say more than 3), assume it's the start of the story
        if len(line.split()) > 3:
            story_started = True
        # Once we detect the start of the story, append the rest of the lines
        if story_started:
            story_lines.append(line)
    
    # Step 3: Combine the story lines into a single string and return the result
    return ' '.join(story_lines).strip()

In [4]:
data['STORY TEXT'] = data['STORY TEXT'].apply(extract_story)

In [5]:
# we have
# 3414 of stories
data

,STORY ID,STORY TEXT
0,1.0,Ludwig Wörl (1906-1967) spent 11 years of his ...
1,2.0,Johann Pscheidt was born in the city of Radaut...
2,3.0,"Even before the war, Professor Tadeusz Czeżows..."
3,4.0,"During the occupation, Władysław Kowalski, a q..."
4,6.0,"Władysława Choms, the wife of a major in the P..."
...,...,...
3409,13524.0,"Mordechai Wulkan (b. 1910) and his wife, Chaja..."
3410,13611.0,"Sala Armel-Goldhar lived with her parents, Isr..."
3411,5257.0,Following Helena Zienowicz’s graduation from t...
3412,3053.0,"Before the war, Badowski Stefan Franciszek liv..."


# 1. Milestone 1: Find Names - NER & Co-reference Resolution

## 1.1 Find all Names: Simple NER: Start with SPACY Package and model “en_core_web_md” + "en_coreference_web_trf"

In [6]:
import spacy
import neuralcoref
from spacy import displacy

In [33]:
# !python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 26.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [7]:
# Load SpaCy's English model
nlp = spacy.load('en_core_web_md')

# Add neuralcoref to SpaCy's pipeline
neuralcoref.add_to_pipe(nlp)

## Coreference Resolution

In [8]:
# import tqdm to track a progress
from tqdm import tqdm
tqdm.pandas()

In [9]:
data['STORY TEXT'] = data['STORY TEXT'].progress_apply(lambda row : nlp(row)._.coref_resolved)
data.to_csv('./drafts/3415_all_coref_data.csv', index = False)

100%|██████████| 3414/3414 [1:39:23<00:00,  1.75s/it]    


In [ ]:
data = pd.read_csv('/drafts/3415_all_coref_data.csv')

### Separate into sentences, find Named Entity (PERSON) and make a dataframe

In [15]:
# Create an empty list to store results
results = []

# Function to extract PERSON entities and their sentences from each story
def extract_person_and_sentence(row):
    story_id = row['STORY ID']
    story_text = row['STORY TEXT']
    
    # Parse the story text using Spacy
    doc = nlp(story_text)
    
    # For each sentence in the document, check for PERSON entities and store the result
    for sentence in doc.sents:
        for entity in sentence.ents:
            if entity.label_ == 'PERSON':
                results.append({
                    'storynumber': story_id,
                    'name': entity.text,
                    'sentence': sentence.text
                })

In [16]:
# Apply the function to each row in the DataFrame
data.progress_apply(extract_person_and_sentence, axis=1)

  3%|▎         | 114/3414 [01:39<48:02,  1.14it/s] 


KeyboardInterrupt: 

In [18]:
# Convert the list of results to a DataFrame
table_1 = pd.DataFrame(results)

In [11]:
for id, story in zip(data['STORY ID'], data['STORY TEXT']):
    sentences = [i for i in nlp(story).sents]
    for sentence in sentences:
        for entity in sentence.ents:
            if entity.label_ == 'PERSON':
                stc = str(sentence)
                new_row = {'storynumber' : id, 'name' : entity.text, 'sentence' : stc}
                table_1.loc[len(table_1)] = new_row

KeyboardInterrupt: 

In [124]:
table_1

,storynumber,name,sentence
0,1.0,Ludwig Wörl,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
1,1.0,Ludwig Wörl,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
2,1.0,Ludwig Wörl,Ludwig Wörl (1906-1967) was first arrested by ...
3,1.0,Ludwig Wörl,After spending some nine months in a dark dete...
4,1.0,Ludwig Wörl,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
...,...,...,...
591,33.0,Strauss,The Jewish pharmacist Strauss was likewise ass...
592,33.0,Strauss,The Jewish pharmacist Strauss was likewise ass...
593,33.0,Werner Krumme,Werner Krumme himself was released from Auschw...
594,33.0,Vashem,"On June 16, 1964, Yad Vashem recognized Werner..."


In [110]:
# Extract Output
table_1.to_csv('./output/3415_SPACY_All_Names.xlsx', index = False)

In [68]:
table_1 = pd.read_csv('./output/3415_SPACY_All_Names.xlsx')

## 1.2 Find Rescuers Names: Filtering Only Rescuers (Righteous Among the Nations) Names

### 1.2.1.1 Step 1: Preparing a table of filtered sentences: “Rescuing Phrases-Name” - Lexicon-based

In [125]:
# Table 1
table_1
# table1 = pd.read_excel('./output/3415_SPACY_All_Names.xlsx', index = False)

,storynumber,name,sentence
0,1.0,Ludwig Wörl,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
1,1.0,Ludwig Wörl,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
2,1.0,Ludwig Wörl,Ludwig Wörl (1906-1967) was first arrested by ...
3,1.0,Ludwig Wörl,After spending some nine months in a dark dete...
4,1.0,Ludwig Wörl,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
...,...,...,...
591,33.0,Strauss,The Jewish pharmacist Strauss was likewise ass...
592,33.0,Strauss,The Jewish pharmacist Strauss was likewise ass...
593,33.0,Werner Krumme,Werner Krumme himself was released from Auschw...
594,33.0,Vashem,"On June 16, 1964, Yad Vashem recognized Werner..."


In [126]:
# Table 2 (lexicon)
table_2 = pd.read_excel('./data/Final_Lexicon_Updated.xlsx')

In [127]:
table_2

,Story ID,Rescuing Verb,Rescuing Phrase
0,1.0,sent,his life as a political prisoner in nazi conce...
1,1.0,sent,"to the camp’s joinery and later, as a trained ..."
2,1.0,employed,personnel. appointed as the lagerälteste (the ...
3,1.0,saving,lagerälteste (the camp elder) of the hospital ...
4,1.0,put himself at risk,"barracks, wörl, against the express orders of ..."
...,...,...,...
8922,5257.0,looked after,"Helena looked after the children, obtaining ""A..."
8923,5257.0,looked after,"Helena looked after the children, obtaining ""A..."
8924,5257.0,sheltered,"Initially, they were only supposed to stay wit..."
8925,5257.0,obtained,Helenapresented the fugitives as her brother’s...


#### Trying rescuing verb 등장하는 부분을 lexicon 의 Rescuing Phrase 활용해서 Story 에서 찾기 중인데 쉽지않음

In [98]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/minjoo/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [118]:
# Step 1: Merge the two dataframes on 'STORY ID'
merged_df = pd.merge(table_2, data, on='Story ID')

In [119]:
# Step 2: Function to find and replace 'Rescuing Phrase' with the best match in 'STORY TEXT'
def update_rescuing_phrase(row):
    story_text = row['Story Text']
    rescuing_phrase = row['Rescuing Phrase']
    
    # Split the STORY TEXT into sentences
    sentences = story_text.split('. ')
    
    # Split the Rescuing Phrase into its own sentences for matching
    rescuing_sentences = rescuing_phrase.split('. ')
    
    # Track which sentences should be updated
    updated_sentences = []
    
    # Iterate through each rescuing sentence and match it to sentences in the story
    for resc_sentence in rescuing_sentences:
        match = process.extractOne(resc_sentence, sentences, scorer=fuzz.partial_ratio)
        
        # If a good match is found, replace the matched sentence with the rescuing sentence
        if match and match[1] > 80:  # You can adjust the threshold as needed
            matched_sentence = match[0]
            updated_sentences.append(matched_sentence)
            sentences[sentences.index(matched_sentence)] = resc_sentence  # Replace in STORY TEXT
            
    # Join the updated sentences back together
    return '. '.join(sentences)

In [120]:
# Step 3: Apply the function to each row to update the 'Rescuing Phrase'
merged_df['Updated Rescuing Phrase'] = merged_df.apply(update_rescuing_phrase, axis=1)

In [121]:
merged_df

,Story ID,Rescuing Verb,Rescuing Phrase,Story Text,Updated Rescuing Phrase
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...,his life as a political prisoner in nazi conce...
1,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
2,1.0,employed,personnel. appointed as the lagerälteste (the ...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
3,1.0,saving,lagerälteste (the camp elder) of the hospital ...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
4,1.0,put himself at risk,"barracks, wörl, against the express orders of ...",Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
...,...,...,...,...,...
84,29.0,sent,"feiner, the bund representative and adolf berm...","From September 1942, even before the establish...",the. When Zegota was established in December 1...
85,29.0,helping,"in december 1942, bartoszewski (whose undercov...","From September 1942, even before the establish...","From September 1942, even before the establish..."
86,29.0,helped,the polish public. he published many articles ...,"From September 1942, even before the establish...","From September 1942, even before the establish..."
87,30.0,sent,"the world looks on”), describing the warsaw gh...","During the occupation, Kann organized the Scou...","During the occupation, Kann organized the Scou..."


In [122]:
merged_df.iloc[-1]['Rescuing Phrase']

'his wife were held in prison in germany until january 31, 1943, and then deported to auschwitz. mrs. krumme was sent to the gas chambers. an ss-man notified werner of her death, in may 1943, with the following words: “be happy'

In [123]:
merged_df.iloc[-1]['Updated Rescuing Phrase']

'Three months after Hitler’s rise to power, in May 1933, Werner Krumme (b. 1909 in Dortmund) married a Jewish wife. Despite official discrimination and frequent harassment by the Gestapo, Krumme stood by Krumme and refused to divorce a Jewish wife. mrs. Krumme, Renate and Anita Lasker, to flee to unoccupied France. his wife were held in prison in germany until january 31, 1943, and then deported to auschwitz. krumme was sent to the gas chambers. an ss-man notified werner of her death, in may 1943, with the following words: “be happy.  Werner used Werner position to falsify so-called selection lists and to assign Jews to work units with relatively tolerable conditions, which increased the chances of Jews survival. Werner thus added the names of Ernst Krinski and Peter Schwartz to a list of a transport that was leaving Auschwitz to the concentration camp of “Warschau,” where no gassing of prisoners took place. The Jewish pharmacist Strauss was likewise assigned by The Jewish pharmacist S

In [106]:
# Step 2: Function to find and replace 'Rescuing Phrase' in 'STORY TEXT'
def replace_rescuing_phrase(row):
    story_text = row['Story Text']
    rescuing_phrase = row['Rescuing Phrase']
    
    # Find the closest matching sentence in the STORY TEXT using fuzzy matching
    match = process.extractOne(rescuing_phrase, story_text.split('. '), scorer=fuzz.partial_ratio)
    
    # If a match is found, replace it with the rescuing phrase
    if match and match[1] > 80:  # Match score should be high enough, e.g., above 80
        matched_sentence = match[0]
        new_story_text = story_text.replace(matched_sentence, rescuing_phrase)
        return new_story_text
    else:
        return story_text  # If no good match, return the original story text

In [107]:
# Step 3: Apply the function to each row to replace the rescuing phrase
merged_df['Updated STORY TEXT'] = merged_df.apply(replace_rescuing_phrase, axis=1)

In [108]:
merged_df

,Story ID,Rescuing Verb,Rescuing Phrase,Story Text,Updated STORY TEXT
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
1,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
2,1.0,employed,personnel. appointed as the lagerälteste (the ...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
3,1.0,saving,lagerälteste (the camp elder) of the hospital ...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
4,1.0,put himself at risk,"barracks, wörl, against the express orders of ...",Ludwig Wörl (1906-1967) spent 11 years of Ludw...,Ludwig Wörl (1906-1967) spent 11 years of Ludw...
...,...,...,...,...,...
84,29.0,sent,"feiner, the bund representative and adolf berm...","From September 1942, even before the establish...","From September 1942, even before the establish..."
85,29.0,helping,"in december 1942, bartoszewski (whose undercov...","From September 1942, even before the establish...","From September 1942, even before the establish..."
86,29.0,helped,the polish public. he published many articles ...,"From September 1942, even before the establish...","From September 1942, even before the establish..."
87,30.0,sent,"the world looks on”), describing the warsaw gh...","During the occupation, Kann organized the Scou...","During the occupation, Kann organized the Scou..."


### Making Table 3 Starts

In [128]:
#Table3
table_3 = pd.DataFrame(columns = ['Story ID','Rescuing Verb', 'Rescuing_phrases-name'])

In [28]:
# input
# Table1
table_1['sentence'][0]
# check if rescuing verb appear in the sentence. (Lexicon reference - same story ID)
# If there is rescuing verb existed,
# (Also name)
# add this sentence to a new table Table3 

'Ludwig Wörl (1906-1967) spent 11 years of Ludwig Wörl (1906-1967) life as a political prisoner in Nazi concentration camps.  '

In [134]:
for row in tqdm(table_1.itertuples(), total = table_1.shape[0]):
    # storyid = row.storynumber
    # 본 회차 스토리에서 찾아내야하는 verb 목록 불러오고
    verbs_to_find = list(set(table_2[table_2['Story ID'] == row.storynumber]['Rescuing Verb']))

    for verb in verbs_to_find:
        if verb in row.sentence:
            new_row = {'Story ID' : row.storynumber, 'Rescuing Verb' : str(verb), 'Rescuing_phrases-name' : row.sentence}
            table_3.loc[len(table_3)] = new_row

100%|██████████| 596/596 [00:00<00:00, 1757.99it/s]


In [135]:
table_3

,Story ID,Rescuing Verb,Rescuing_phrases-name
0,1.0,sent,Ludwig Wörl (1906-1967) was first arrested by ...
1,1.0,sent,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
2,1.0,employed,Appointed as the Lagerälteste (the camp elder)...
3,1.0,saving,Appointed as the Lagerälteste (the camp elder)...
4,1.0,employed,Appointed as the Lagerälteste (the camp elder)...
...,...,...,...
177,29.0,sent,"As part of Ludwik”) underground activity, Wład..."
178,29.0,helped,"The author, Rachel Auerbach, and Dr. A. Berman..."
179,29.0,helped,"The author, Rachel Auerbach, and Dr. A. Berman..."
180,30.0,sent,"The tract, which was also sent abroad, also co..."


In [150]:
# Table_4 (Merge table_2 and table_3)
# table_4: Complete Lexicon Structure Example
table_4 = pd.merge(table_2, table_3, how = 'left', on = ['Story ID','Rescuing Verb'])

In [151]:
table_4

,Story ID,Rescuing Verb,Rescuing Phrase,Rescuing_phrases-name
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) was first arrested by ...
1,1.0,sent,his life as a political prisoner in nazi conce...,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
2,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) was first arrested by ...
3,1.0,sent,"to the camp’s joinery and later, as a trained ...","In 1942, Ludwig Wörl (1906-1967) was sent to A..."
4,1.0,employed,personnel. appointed as the lagerälteste (the ...,Appointed as the Lagerälteste (the camp elder)...
...,...,...,...,...
9168,5257.0,looked after,"Helena looked after the children, obtaining ""A...",NaN
9169,5257.0,looked after,"Helena looked after the children, obtaining ""A...",NaN
9170,5257.0,sheltered,"Initially, they were only supposed to stay wit...",NaN
9171,5257.0,obtained,Helenapresented the fugitives as her brother’s...,NaN


In [152]:
# drop duplicated rows
table_4.drop_duplicates(inplace = True)

In [153]:
table_4

,Story ID,Rescuing Verb,Rescuing Phrase,Rescuing_phrases-name
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) was first arrested by ...
1,1.0,sent,his life as a political prisoner in nazi conce...,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
2,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) was first arrested by ...
3,1.0,sent,"to the camp’s joinery and later, as a trained ...","In 1942, Ludwig Wörl (1906-1967) was sent to A..."
4,1.0,employed,personnel. appointed as the lagerälteste (the ...,Appointed as the Lagerälteste (the camp elder)...
...,...,...,...,...
9167,4481.0,recognized,"On January 17, 1990, Yad Vashem recognized Józ...",NaN
9168,5257.0,looked after,"Helena looked after the children, obtaining ""A...",NaN
9170,5257.0,sheltered,"Initially, they were only supposed to stay wit...",NaN
9171,5257.0,obtained,Helenapresented the fugitives as her brother’s...,NaN


In [201]:
table_4.to_csv('./drafts/table_4.xlsx', index = False)

### 1.2.1.2 Step 2:“Rescuers Names filtering” - Step-by-Step Breakdown

In [265]:
table_4 = pd.read_csv('./drafts/table_4.xlsx')

In [178]:
table_4

,Story ID,Rescuing Verb,Rescuing Phrase,Rescuing_phrases-name
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) was first arrested by ...
1,1.0,sent,his life as a political prisoner in nazi conce...,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
2,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) was first arrested by ...
3,1.0,sent,"to the camp’s joinery and later, as a trained ...","In 1942, Ludwig Wörl (1906-1967) was sent to A..."
4,1.0,employed,personnel. appointed as the lagerälteste (the ...,Appointed as the Lagerälteste (the camp elder)...
...,...,...,...,...
9167,4481.0,recognized,"On January 17, 1990, Yad Vashem recognized Józ...",NaN
9168,5257.0,looked after,"Helena looked after the children, obtaining ""A...",NaN
9170,5257.0,sheltered,"Initially, they were only supposed to stay wit...",NaN
9171,5257.0,obtained,Helenapresented the fugitives as her brother’s...,NaN


#### 0917 Dependency Parsing.. hmm

In [235]:
table_4.dropna(inplace = True)

In [204]:
table_4.iloc[0]

Story ID                                                               1.0
Rescuing Verb                                                         sent
Rescuing Phrase          his life as a political prisoner in nazi conce...
Rescuing_phrases-name    Ludwig Wörl (1906-1967) was first arrested by ...
Name: 0, dtype: object

In [223]:
table_4.iloc[4]['Rescuing_phrases-name']

'Appointed as the Lagerälteste (the camp elder) of the hospital barracks, Wörl, against the express orders of the SS, employed Jewish doctors, thus saving Jewish doctors from certain death.'

In [206]:
txt = 'Ludwig Wörl (1906-1967) was first arrested by the Gestapo in 1934, and sent to Dachau for distributing a pamphlet in which the citizens of Munich were informed about the horrors of the camp.'


In [207]:
doc = nlp(txt)

In [212]:
for token in doc :
    if token.text.lower() == 'sent':
        print(token.dep_)
        print([token for token in token.children])
        print([token.dep_ for token in token.children])

conj
[to, for]
['dative', 'prep']


In [ ]:
# 09-17 !!!!!!!!
# Function to find the rescuer (subject) for each sentence
def find_rescuer(df):
    sentence = df['Rescuing Phrase']
    rescuing_verb = df['Rescuing Verb']
    
    # Skip NaN values
    if pd.isna(sentence):
        return None
    
    # Process sentence with Spacy
    doc = nlp(sentence)
    
    # If the verb is multi-word (like "put himself at risk"), we handle it separately.
    if rescuing_verb in sentence:
        # Find the subject (nsubj) related to the rescuing verb
        for token in doc:
            if rescuing_verb.split()[0] == token.lemma_ or token.text == rescuing_verb.split()[0]:
                # Find the subject (nsubj) linked to the verb
                subject = [child for child in token.children if child.dep_ == "nsubj"]
                if subject:
                    return subject[0].text  # Return the first subject found
    
    return None  # Return None if no subject is found

In [250]:
# 주어진 rescuing verb와 관련된 주어(구조하는 사람) 찾기
def find_rescuer(df):
    # 문장과 rescuing verb
    sentence = df['Rescuing_phrases-name']
    rescuing_verb = df['Rescuing Verb']
    
    # 문장을 Spacy로 파싱
    doc = nlp(sentence)
    
    # 구조하는 사람(주어)을 저장할 리스트
    # rescuers = []
    
    # 문장에서 주어진 rescuing verb의 텍스트와 일치하는 주어 찾기
    # subject = None
    rescuer = None
    for token in doc:
        
        # if token.dep_ == 'nsubj':
        #     subject = token.text

        # token.text가 주어진 rescuing verb와 일치할 경우
        if token.text.lower() == rescuing_verb.lower():  # 대소문자 구분 무시
            # 해당 동사에 연결된 주어 찾기 (nsubj)
            for child in token.children:
                if child.dep_ == "nsubj":
                    rescuer = child.text
    
    # 결과 반환 (찾은 구조하는 사람 또는 None)
    if rescuer:
        return rescuer
    # else:
    #     return subject
    # if rescuers else None

In [251]:
sample_table_4 = table_4[:40]

In [252]:
# 데이터프레임에서 rescuing verb와 관련된 주어(구조하는 사람) 찾기
sample_table_4['Rescuer'] = sample_table_4.apply(find_rescuer, axis=1)

/Users/minjoo/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [253]:
sample_table_4

,Story ID,Rescuing Verb,Rescuing Phrase,Rescuing_phrases-name,Rescuer
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) was first arrested by ...,None
1,1.0,sent,his life as a political prisoner in nazi conce...,"In 1942, Ludwig Wörl (1906-1967) was sent to A...",None
2,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) was first arrested by ...,None
3,1.0,sent,"to the camp’s joinery and later, as a trained ...","In 1942, Ludwig Wörl (1906-1967) was sent to A...",None
4,1.0,employed,personnel. appointed as the lagerälteste (the ...,Appointed as the Lagerälteste (the camp elder)...,None
6,1.0,saving,lagerälteste (the camp elder) of the hospital ...,Appointed as the Lagerälteste (the camp elder)...,None
9,1.0,obtain,"the ss, employed jewish doctors, thus saving t...",Ludwig Wörl (1906-1967) also put Ludwig Wörl (...,None
11,1.0,treatment,himself at risk in order to obtain at least a ...,Ludwig Wörl (1906-1967) also put Ludwig Wörl (...,None
13,1.0,treatment,himself at risk in order to obtain at least a ...,"Here, again, Wörl made Wörl beneficial influen...",None
15,1.0,forge,obtain at least a minimum of the required medi...,Ludwig Wörl (1906-1967) would forge selection ...,Wörl


### 그냥 이걸로 갈까봐... 첫번째 이름 찾기

In [254]:
# Function to find the full name of the rescuer, and split it into first name and last name
def extract_name_info(row):
    sentence = row['Rescuing_phrases-name']  # Adjusted column name
    rescuing_verb = row['Rescuing Verb']    # Adjusted column name
    
    # Skip NaN values
    if pd.isna(sentence):
        return pd.Series([None, None, None])  # Return None for full name, first name, and last name
    
    # Process sentence with Spacy
    doc = nlp(sentence)
    
    # Find the full name of the rescuer using NER
    full_name = None
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            full_name = ent.text  # Get the first PERSON entity (assuming it's the rescuer)
            break
    
    if not full_name:
        return pd.Series([None, None, None])  # Return None if no PERSON entity is found
    
    # Split the full name into first and last name
    name_parts = full_name.split()
    if len(name_parts) > 1:
        last_name = name_parts[-1]
        first_name = " ".join(name_parts[:-1])
    else:
        first_name = full_name
        last_name = None  # If no last name, leave it as None
    
    return pd.Series([full_name, first_name, last_name])

In [255]:
sample_table_4 = table_4[:40]

In [158]:
sample_table_4

,Story ID,Rescuing Verb,Rescuing Phrase,Rescuing_phrases-name
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) was first arrested by ...
1,1.0,sent,his life as a political prisoner in nazi conce...,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
2,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) was first arrested by ...
3,1.0,sent,"to the camp’s joinery and later, as a trained ...","In 1942, Ludwig Wörl (1906-1967) was sent to A..."
4,1.0,employed,personnel. appointed as the lagerälteste (the ...,Appointed as the Lagerälteste (the camp elder)...
6,1.0,saving,lagerälteste (the camp elder) of the hospital ...,Appointed as the Lagerälteste (the camp elder)...
8,1.0,put himself at risk,"barracks, wörl, against the express orders of ...",NaN
9,1.0,obtain,"the ss, employed jewish doctors, thus saving t...",Ludwig Wörl (1906-1967) also put Ludwig Wörl (...
11,1.0,treatment,himself at risk in order to obtain at least a ...,Ludwig Wörl (1906-1967) also put Ludwig Wörl (...
13,1.0,treatment,himself at risk in order to obtain at least a ...,"Here, again, Wörl made Wörl beneficial influen..."


In [256]:
# Apply the function to the DataFrame and store the results in separate columns
sample_table_4[['Full Name', 'First Name', 'Last Name']] = sample_table_4.apply(extract_name_info, axis=1)

/Users/minjoo/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [257]:
sample_table_4

,Story ID,Rescuing Verb,Rescuing Phrase,Rescuing_phrases-name,Full Name,First Name,Last Name
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) was first arrested by ...,Ludwig Wörl,Ludwig,Wörl
1,1.0,sent,his life as a political prisoner in nazi conce...,"In 1942, Ludwig Wörl (1906-1967) was sent to A...",Ludwig Wörl,Ludwig,Wörl
2,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) was first arrested by ...,Ludwig Wörl,Ludwig,Wörl
3,1.0,sent,"to the camp’s joinery and later, as a trained ...","In 1942, Ludwig Wörl (1906-1967) was sent to A...",Ludwig Wörl,Ludwig,Wörl
4,1.0,employed,personnel. appointed as the lagerälteste (the ...,Appointed as the Lagerälteste (the camp elder)...,Lagerälteste,Lagerälteste,None
6,1.0,saving,lagerälteste (the camp elder) of the hospital ...,Appointed as the Lagerälteste (the camp elder)...,Lagerälteste,Lagerälteste,None
9,1.0,obtain,"the ss, employed jewish doctors, thus saving t...",Ludwig Wörl (1906-1967) also put Ludwig Wörl (...,Ludwig Wörl,Ludwig,Wörl
11,1.0,treatment,himself at risk in order to obtain at least a ...,Ludwig Wörl (1906-1967) also put Ludwig Wörl (...,Ludwig Wörl,Ludwig,Wörl
13,1.0,treatment,himself at risk in order to obtain at least a ...,"Here, again, Wörl made Wörl beneficial influen...",Wörl,Wörl,None
15,1.0,forge,obtain at least a minimum of the required medi...,Ludwig Wörl (1906-1967) would forge selection ...,Ludwig Wörl,Ludwig,Wörl


In [161]:
table_5 = sample_table_4[['Story ID','Full Name', 'Last Name', 'First Name', 'Rescuing_phrases-name']]

In [162]:
table_5

,Story ID,Full Name,Last Name,First Name,Rescuing_phrases-name
0,1.0,Ludwig Wörl,Wörl,Ludwig,Ludwig Wörl (1906-1967) was first arrested by ...
1,1.0,Ludwig Wörl,Wörl,Ludwig,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
2,1.0,Ludwig Wörl,Wörl,Ludwig,Ludwig Wörl (1906-1967) was first arrested by ...
3,1.0,Ludwig Wörl,Wörl,Ludwig,"In 1942, Ludwig Wörl (1906-1967) was sent to A..."
4,1.0,Lagerälteste,None,Lagerälteste,Appointed as the Lagerälteste (the camp elder)...
6,1.0,Lagerälteste,None,Lagerälteste,Appointed as the Lagerälteste (the camp elder)...
8,1.0,None,None,None,NaN
9,1.0,Ludwig Wörl,Wörl,Ludwig,Ludwig Wörl (1906-1967) also put Ludwig Wörl (...
11,1.0,Ludwig Wörl,Wörl,Ludwig,Ludwig Wörl (1906-1967) also put Ludwig Wörl (...
13,1.0,Wörl,None,Wörl,"Here, again, Wörl made Wörl beneficial influen..."


In [ ]:
table_5.to_csv('3415_SPACY_All_Rescuers.xlsx', index = False)

### Rescuing DATE

In [164]:
# Function to find the full name of the rescuer, first name, last name, and the rescued date
def extract_name_and_date(row):
    sentence = row['Rescuing_phrases-name']  # Adjusted column name
    rescuing_verb = row['Rescuing Verb']    # Adjusted column name
    
    # Skip NaN values
    if pd.isna(sentence):
        return pd.Series([None, None, None, None])  # Return None for all if sentence is NaN
    
    # Process sentence with Spacy
    doc = nlp(sentence)
    
    # Extract full name (PERSON) and rescued date (DATE)
    full_name = None
    rescued_date = None
    for ent in doc.ents:
        if ent.label_ == "PERSON" and full_name is None:  # Get the first PERSON entity
            full_name = ent.text
        if ent.label_ == "DATE" and rescued_date is None:  # Get the first DATE entity
            rescued_date = ent.text

    if not full_name:
        return pd.Series([None, None, None, rescued_date])  # Return None for names if no PERSON found
    
    # Split the full name into first name and last name
    name_parts = full_name.split()
    if len(name_parts) > 1:
        last_name = name_parts[-1]
        first_name = " ".join(name_parts[:-1])
    else:
        first_name = full_name
        last_name = None  # If no last name, leave it as None
    
    return pd.Series([full_name, first_name, last_name, rescued_date])

In [165]:
# Apply the function to the DataFrame and store the results in separate columns
sample_table_4[['Full Name', 'First Name', 'Last Name', 'Rescued Date']] = sample_table_4.apply(extract_name_and_date, axis=1)

/Users/minjoo/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [166]:
sample_table_4

,Story ID,Rescuing Verb,Rescuing Phrase,Rescuing_phrases-name,Full Name,First Name,Last Name,Rescued Date
0,1.0,sent,his life as a political prisoner in nazi conce...,Ludwig Wörl (1906-1967) was first arrested by ...,Ludwig Wörl,Ludwig,Wörl,1906-1967
1,1.0,sent,his life as a political prisoner in nazi conce...,"In 1942, Ludwig Wörl (1906-1967) was sent to A...",Ludwig Wörl,Ludwig,Wörl,1942
2,1.0,sent,"to the camp’s joinery and later, as a trained ...",Ludwig Wörl (1906-1967) was first arrested by ...,Ludwig Wörl,Ludwig,Wörl,1906-1967
3,1.0,sent,"to the camp’s joinery and later, as a trained ...","In 1942, Ludwig Wörl (1906-1967) was sent to A...",Ludwig Wörl,Ludwig,Wörl,1942
4,1.0,employed,personnel. appointed as the lagerälteste (the ...,Appointed as the Lagerälteste (the camp elder)...,Lagerälteste,Lagerälteste,None,None
6,1.0,saving,lagerälteste (the camp elder) of the hospital ...,Appointed as the Lagerälteste (the camp elder)...,Lagerälteste,Lagerälteste,None,None
8,1.0,put himself at risk,"barracks, wörl, against the express orders of ...",NaN,None,None,None,None
9,1.0,obtain,"the ss, employed jewish doctors, thus saving t...",Ludwig Wörl (1906-1967) also put Ludwig Wörl (...,Ludwig Wörl,Ludwig,Wörl,1906-1967
11,1.0,treatment,himself at risk in order to obtain at least a ...,Ludwig Wörl (1906-1967) also put Ludwig Wörl (...,Ludwig Wörl,Ludwig,Wörl,1906-1967
13,1.0,treatment,himself at risk in order to obtain at least a ...,"Here, again, Wörl made Wörl beneficial influen...",Wörl,Wörl,None,None


In [ ]:
table_6.to_csv('3415_SPACY_All_Rescuers_Dates.xlsx', index = False)

In [94]:
# 문장 처리
# doc = nlp(sentence)

# 사람(Entity) 추출
people = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
print("Detected people:", people)

# 의존성 파싱으로 주어-동사 관계 분석
for token in doc:
    if token.dep_ == "nsubj" and token.head.lemma_ == "help":
        print(f"Subject: {token.text}, Verb: {token.head.text}")

Detected people: ['Vashem', 'Werner Krumme']


In [ ]:
3415_SPACY_All_Rescuers.XLS

In [ ]:
output_file_name = '3415_SPACY_All_Names.xlsx'
output.to_csv(f'./output/{output_file_name}')